Download and unzip fer2013.csv

In [4]:
!tar xvzf fer2013.tar.gz
!mv fer2013/fer2013.csv .
!rm -rf fer2013

fer2013/fer2013.csv
fer2013/README
fer2013/fer2013.bib
fer2013/


Extract images from fer2013.csv 

In [ ]:
import random
import os
import cv2
import pandas as pd
import numpy as np

curdir = '..'
outputdir=os.path.join(curdir,'Data','Human')
def expression(argument):
    switcher = {
        0:"Angry", 
        1:"Disgust", 
        2:"Fear", 
        3:"Happy", 
        4:"Sad", 
        5:"Surprise", 
        6:"Neutral"
    }
    return switcher.get(argument, "Invalid number")

def gen_record(csvfile):
    data = pd.read_csv(csvfile, delimiter=',', dtype='a')
    labels = np.array(data['emotion'], np.float)
    imagebuffer = np.array(data['pixels'])
    images = np.array([np.fromstring(image, np.uint8, sep=' ') for image in imagebuffer])
    del imagebuffer
    num_shape = int(np.sqrt(images.shape[-1]))
    images.shape = (images.shape[0], num_shape, num_shape)
    #img=images[0];cv2.imshow('test',img);cv2.waitKey(0);cv2.destroyAllWindow();exit()
    dirs = set(data['Usage'])
    subdirs = set(labels)
    class_dir = {}
    for dr in dirs:
        dest = os.path.join(outputdir, dr)
        class_dir[dr] = dest
        if not os.path.exists(dest):
            os.mkdir(dest)
    data = zip(labels, images, data['Usage'])

    for d in data:
        destdir = os.path.join(class_dir[d[-1]], expression(int(d[0])))
        if not os.path.exists(destdir):
            os.mkdir(destdir)
        img = d[1]
        filepath = unique_name(destdir, d[-1])
        print('[^_^] Write image to %s' % filepath)
        if not filepath:
            continue
        sig = cv2.imwrite(filepath, img)
        if not sig:
            print('Error')
            exit(-1)


def unique_name(pardir, prefix, suffix='jpg'):
    filename = '{0}_{1}.{2}'.format(prefix, random.randint(1, 10 ** 8), suffix)
    filepath = os.path.join(pardir, filename)
    if not os.path.exists(filepath):
        return filepath
    unique_name(pardir, prefix, suffix)
    
def create_dir(path):
    if not os.path.exists(path):
        os.mkdir(path)

print('start')
create_dir('../Data')
create_dir('../Data/Human')
filename = 'fer2013.csv'
gen_record(filename)
print('done')

Change 48 * 48 images to 256 * 256

In [7]:
import os
import glob
from PIL import Image
curpath='..'
pathfrom = os.path.join(curpath,'Data','Human','Training')
pathto =os.path.join(curpath,'Data','Human','resize')
if not os.path.exists(pathto):
    os.mkdir(pathto)
dirs = os.listdir(pathfrom)
for d in dirs:
    destdir=os.path.join(pathto,d)
    if not os.path.exists(destdir):
        os.mkdir(destdir)
    for root, dirs, files in os.walk(os.path.join(pathfrom, d)):
        for f in files:
            im = Image.open(os.path.join(pathfrom,d,f))
            im2 = im.resize((256, 256), Image.BICUBIC)
            im2.save(os.path.join(destdir,f))
print('done')

done
